# Week 1 — Exploratory Data Analysis (EDA) on C-MAPSS FD001

**Learning Goals:**
- Understand the C-MAPSS dataset structure (units, cycles, settings, sensors)
- Understand RUL labeling and common pitfalls (leakage, scaling, windowing)
- Identify informative vs non-informative sensors

**Dataset:** NASA C-MAPSS FD001 — 100 train engines, 100 test engines, 1 operating condition, HPC degradation

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

# Project imports
import sys
sys.path.insert(0, '../../src')
from data.data_loader import load_train, load_test, load_all, SENSOR_COLS, SETTING_COLS, INDEX_COLS, ALL_COLS

print('All imports successful!')

## 1. Load the Data

In [ ]:
# Load FD001 train data (with RUL labels, capped at 125)
df_train = load_train(fd_number=1, rul_cap=125)
df_test, rul_true = load_test(fd_number=1)

print(f'Training data shape: {df_train.shape}')
print(f'Test data shape:     {df_test.shape}')
print(f'Test RUL values:     {len(rul_true)} engines')
print(f'\nColumn names: {list(df_train.columns)}')

In [ ]:
# Quick look at first few rows
df_train.head(10)

In [ ]:
# Summary statistics
df_train.describe()

## 2. Dataset Structure Overview

In [ ]:
# Number of engines and cycles
n_engines = df_train['unit_id'].nunique()
cycles_per_engine = df_train.groupby('unit_id')['cycle'].max()

print(f'Number of training engines: {n_engines}')
print(f'\nCycles per engine (= engine lifetime):')
print(f'  Min:    {cycles_per_engine.min()}')
print(f'  Max:    {cycles_per_engine.max()}')
print(f'  Mean:   {cycles_per_engine.mean():.1f}')
print(f'  Median: {cycles_per_engine.median():.1f}')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of engine lifetimes
axes[0].hist(cycles_per_engine, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(cycles_per_engine.mean(), color='red', linestyle='--', label=f'Mean: {cycles_per_engine.mean():.0f}')
axes[0].set_xlabel('Engine Lifetime (cycles)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Engine Lifetimes (FD001 Train)')
axes[0].legend()

# Sorted lifetimes bar chart
sorted_cycles = cycles_per_engine.sort_values()
axes[1].bar(range(len(sorted_cycles)), sorted_cycles.values, color='steelblue', alpha=0.7)
axes[1].set_xlabel('Engine (sorted by lifetime)')
axes[1].set_ylabel('Lifetime (cycles)')
axes[1].set_title('Engine Lifetimes — Sorted')

plt.tight_layout()
plt.show()

## 3. RUL Label Analysis

In [ ]:
# RUL distribution in training data
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RUL distribution (all training rows)
axes[0].hist(df_train['RUL'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0].set_xlabel('RUL (capped at 125)')
axes[0].set_ylabel('Count')
axes[0].set_title('RUL Distribution — All Training Rows')

# True test RUL distribution
axes[1].hist(rul_true, bins=20, edgecolor='black', alpha=0.7, color='seagreen')
axes[1].set_xlabel('True RUL')
axes[1].set_ylabel('Count')
axes[1].set_title('True Test RUL Distribution (FD001)')

# RUL over cycle for a few engines
for unit_id in [1, 25, 50, 75, 100]:
    unit = df_train[df_train['unit_id'] == unit_id]
    axes[2].plot(unit['cycle'], unit['RUL'], label=f'Engine {unit_id}', alpha=0.8)
axes[2].set_xlabel('Cycle')
axes[2].set_ylabel('RUL (capped at 125)')
axes[2].set_title('RUL Labeling — Sample Engines')
axes[2].legend(fontsize=9)

plt.tight_layout()
plt.show()

print('Note: RUL is capped at 125 (piece-wise linear). Early cycles have RUL=125.')

## 4. Operational Settings Analysis

In [ ]:
# In FD001, there's only 1 operating condition — settings should be near-constant
print('Operational settings statistics (FD001 = 1 operating condition):')
print(df_train[SETTING_COLS].describe())
print()
print('Unique values per setting:')
for col in SETTING_COLS:
    print(f'  {col}: {df_train[col].nunique()} unique values, range [{df_train[col].min():.4f}, {df_train[col].max():.4f}]')

In [ ]:
# Visualize operational settings
fig, axes = plt.subplots(1, 3, figsize=(18, 4))
for i, col in enumerate(SETTING_COLS):
    axes[i].hist(df_train[col], bins=50, edgecolor='black', alpha=0.7, color='mediumpurple')
    axes[i].set_title(f'{col}')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Count')
plt.suptitle('Operational Settings Distribution (FD001 — should be near-constant)', y=1.02)
plt.tight_layout()
plt.show()

## 5. Sensor Distributions

In [ ]:
# Per-sensor distribution histograms (3x7 grid for 21 sensors)
fig, axes = plt.subplots(3, 7, figsize=(24, 10))

for idx, col in enumerate(SENSOR_COLS):
    ax = axes[idx // 7, idx % 7]
    ax.hist(df_train[col], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax.set_title(col, fontsize=10)
    ax.tick_params(labelsize=8)

plt.suptitle('Sensor Value Distributions (FD001 Train)', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Identify near-constant sensors (low variance)
sensor_variance = df_train[SENSOR_COLS].var().sort_values()

fig, ax = plt.subplots(figsize=(12, 5))
colors = ['red' if v < 1e-4 else 'steelblue' for v in sensor_variance.values]
sensor_variance.plot(kind='bar', ax=ax, color=colors, edgecolor='black', alpha=0.8)
ax.set_ylabel('Variance')
ax.set_title('Sensor Variance (red = near-zero → drop these)')
ax.set_yscale('log')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# List sensors to drop
low_var_sensors = sensor_variance[sensor_variance < 1e-4].index.tolist()
print(f'\nSensors with near-zero variance (to drop): {low_var_sensors}')
print(f'Remaining informative sensors: {[s for s in SENSOR_COLS if s not in low_var_sensors]}')

## 6. Sensor Trends Over Engine Lifetime (Degradation Patterns)

In [ ]:
# Plot sensor trends for a few engines — this is the most important EDA plot!
informative_sensors = ['sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8',
                        'sensor_9', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14',
                        'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21']

sample_engines = [1, 50, 100]

fig, axes = plt.subplots(len(informative_sensors), 1, figsize=(16, 3 * len(informative_sensors)))

for idx, sensor in enumerate(informative_sensors):
    ax = axes[idx]
    for eng_id in sample_engines:
        unit = df_train[df_train['unit_id'] == eng_id]
        ax.plot(unit['cycle'], unit[sensor], label=f'Engine {eng_id}', alpha=0.7)
    ax.set_ylabel(sensor)
    ax.legend(fontsize=8, loc='upper right')
    if idx == 0:
        ax.set_title('Sensor Trends Over Engine Lifetime (Healthy → Degraded)', fontsize=14)
    if idx == len(informative_sensors) - 1:
        ax.set_xlabel('Cycle')

plt.tight_layout()
plt.show()

In [ ]:
# Normalized sensor trends (to compare shapes)
from sklearn.preprocessing import MinMaxScaler

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for ax, eng_id in zip(axes, sample_engines):
    unit = df_train[df_train['unit_id'] == eng_id].copy()
    scaler = MinMaxScaler()
    unit[informative_sensors] = scaler.fit_transform(unit[informative_sensors])
    
    for sensor in informative_sensors:
        ax.plot(unit['cycle'], unit[sensor], label=sensor, alpha=0.6, linewidth=0.8)
    ax.set_title(f'Engine {eng_id} — All Informative Sensors (Normalized 0-1)')
    ax.set_ylabel('Normalized Value')
    ax.legend(fontsize=7, ncol=4, loc='upper left')

axes[-1].set_xlabel('Cycle')
plt.tight_layout()
plt.show()

## 7. Correlation Analysis

In [ ]:
# Correlation heatmap: sensors vs RUL
corr_cols = informative_sensors + ['RUL']
corr_matrix = df_train[corr_cols].corr()

fig, ax = plt.subplots(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, ax=ax, square=True)
ax.set_title('Correlation Matrix: Informative Sensors + RUL')
plt.tight_layout()
plt.show()

# Rank sensors by absolute correlation with RUL
rul_corr = corr_matrix['RUL'].drop('RUL').abs().sort_values(ascending=False)
print('Sensor correlation with RUL (absolute):')
print(rul_corr.to_string())

In [ ]:
# Scatter plots: top 6 sensors vs RUL
top_6_sensors = rul_corr.head(6).index.tolist()

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for idx, sensor in enumerate(top_6_sensors):
    ax = axes[idx // 3, idx % 3]
    ax.scatter(df_train[sensor], df_train['RUL'], alpha=0.1, s=1, c='steelblue')
    ax.set_xlabel(sensor)
    ax.set_ylabel('RUL')
    corr_val = df_train[sensor].corr(df_train['RUL'])
    ax.set_title(f'{sensor} vs RUL (r={corr_val:.3f})')

plt.suptitle('Top 6 Sensors by RUL Correlation', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## 8. Sensor Behavior at Different RUL Stages

In [ ]:
# Compare sensor values at different RUL stages
df_train['rul_stage'] = pd.cut(df_train['RUL'], bins=[0, 30, 80, 125], labels=['Critical (<30)', 'Warning (30-80)', 'Healthy (>80)'])

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for idx, sensor in enumerate(top_6_sensors):
    ax = axes[idx // 3, idx % 3]
    for stage, color in zip(['Healthy (>80)', 'Warning (30-80)', 'Critical (<30)'], ['green', 'orange', 'red']):
        data = df_train[df_train['rul_stage'] == stage][sensor]
        ax.hist(data, bins=40, alpha=0.5, label=stage, color=color, density=True)
    ax.set_title(sensor)
    ax.set_xlabel('Value')
    ax.legend(fontsize=8)

plt.suptitle('Sensor Distributions by RUL Stage', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## 9. Test Data Overview

In [ ]:
# Test data: engines are cut off before failure
test_cycles = df_test.groupby('unit_id')['cycle'].max()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(test_cycles, bins=20, edgecolor='black', alpha=0.7, color='darkorange')
axes[0].set_xlabel('Observed Cycles (before cutoff)')
axes[0].set_ylabel('Count')
axes[0].set_title('Test Engines — Observed Lifetime')

axes[1].hist(rul_true, bins=20, edgecolor='black', alpha=0.7, color='seagreen')
axes[1].set_xlabel('True RUL')
axes[1].set_ylabel('Count')
axes[1].set_title('Test Engines — True Remaining Life')

plt.tight_layout()
plt.show()

print(f'Test engines: {len(rul_true)}')
print(f'True RUL: min={rul_true.min()}, max={rul_true.max()}, mean={rul_true.mean():.1f}')

## 10. Key Takeaways & Decisions

### Findings:
1. **FD001** has 100 train/test engines with a single operating condition
2. **7 sensors** are near-constant (no predictive value) → drop them
3. **14 sensors** show degradation trends → use these for modeling
4. **RUL capping at 125** is standard: early cycles are all "healthy"
5. **Sensor trends** clearly show degradation patterns over engine lifetime
6. **Strong correlations** exist between certain sensors and RUL

### Decisions for Next Steps:
- Drop sensors: sensor_1, 5, 6, 10, 16, 18, 19
- Use RUL cap = 125 as default
- Window size for sequence models: start with 30 cycles
- Scaling: MinMax to [0,1] (fit on train only)

In [ ]:
# Save EDA summary stats for reference
eda_summary = {
    'n_train_engines': n_engines,
    'n_test_engines': len(rul_true),
    'avg_lifetime': float(cycles_per_engine.mean()),
    'informative_sensors': informative_sensors,
    'dropped_sensors': low_var_sensors,
    'rul_cap': 125,
    'top_correlated_sensors': rul_corr.head(6).to_dict(),
}

import json
with open('../../reports/eda_summary.json', 'w') as f:
    json.dump(eda_summary, f, indent=2)

print('EDA summary saved to reports/eda_summary.json')
print(json.dumps(eda_summary, indent=2))